#### Importación de librerias

In [274]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pycountry
import re
import unicodedata
import requests


#### Cargue de datos

In [275]:
data = pd.read_csv('data/raw_data.csv',delimiter=',')


In [276]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28115 entries, 0 to 28114
Data columns (total 18 columns):
 #   Column                                                                                                                                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                --------------  -----  
 0   Timestamp                                                                                                                                                                                                                             28115 non-null  object 
 1   How old are you?                                                                                             

El conjunto de datos proporcionado tiene 28.115 registros y 18 columnas, de las cuales 16 son de tipo categórico y 2 de tipo numérico (salario anual y compesaciones adicionales). 

#### Limpieza de datos

In [277]:
# Cambiar el nombre de las columnas a español y nombres abreviados
data.columns = ['fecha',
                'edad',
                'industria',
                'titulo_profesional',
                'extra_titulo',
                'salario_anual',
                'compensaciones',
                'moneda',
                'otra_moneda',
                'ingresos_extra',
                'pais',
                'estado',
                'ciudad',
                'años_experiencia_total',
                'años_experiencia_industria',
                'nivel_educacion',
                'género',
                'raza'
                ]



In [284]:
data['salario_anual'] = data['salario_anual'].str.replace(',', '', regex=True).astype(float)
data['compensaciones'] = data['compensaciones'].astype(float)

In [278]:

# Asegurar que las columnas de Pais y Ciudad sean string
data['pais'] = data['pais'].astype(str)
data['ciudad'] = data['ciudad'].astype(str)

In [279]:
def estandarizar_pais(nombre):
    nombre = nombre.strip().lower()
    if re.findall(r"u.s|states|us|america|u. s|u. s.|u.a.|u.s|u.s.|u.sa|ua|u.s.a.|usa|uxz|united|isa|uniter statez|virginia|hartford|san francisco|california", nombre):
        return 'USA'
    elif re.findall(r"united kingdom|uk|kingdom|u.k|great britain|britain|wales|europe|london|scotland|Isle of Man|Cayman Islands",nombre):
        return 'UK'
    elif re.findall(r"canada|canadw|csnada|canda|canad",nombre):
        return 'Canada'
    elif re.findall(r"hong kong|hong konh",nombre):
        return 'Hong Kong'
    elif re.findall(r"netherlands|nederland",nombre):
        return 'Netherlands'
    elif re.findall(r"england|englang",nombre):
        return 'England'
    elif re.findall(r"ireland|loutreland",nombre):
        return 'Ireland'
    elif re.findall(r"spain|catalonia",nombre):
        return 'Spain'
    elif re.findall(r"italy|italia",nombre):
        return 'Italy'
    elif re.findall(r"denmark|danmark",nombre):
        return 'Denmark'
    elif re.findall(r"vietnam|viet nam",nombre):
        return 'Vietnam'
    elif re.findall(r"panama|panamá",nombre):
        return 'Panama'
    elif re.findall(r"brazil|brasil",nombre):
        return 'Brazil'
    elif re.findall(r"mexico|méxico",nombre):
        return 'Mexico'
    elif re.findall(r"india|ibdia",nombre):
        return 'India'
    elif re.findall(r"myanmar|burma",nombre):
        return 'Myanmar'
    elif re.findall(r"czechia|česká republika",nombre):
        return 'Czechia'
    elif re.findall(r"bahamas",nombre):
        return 'Bahamas'
    elif re.findall(r"d'ivoire",nombre):
        return "Côte d'Ivoire"
    elif re.findall(r"china",nombre):
        return 'China'
    elif re.findall(r"africa",nombre):
        return 'Africa'
    elif re.findall(r"luxemburg",nombre):
        return 'Luxembourg'
    elif re.findall(r"other|global|contracts|i.s|remote|international|policy|dbfemf",nombre) or len(nombre)>15 or len(nombre)<3:
        return 'Other'    
    else:
        try:
            return pycountry.countries.lookup(nombre).name
        except LookupError:
            return nombre  

# Cargar el conjunto de datos de ciudades del mundo
cities_df = pd.read_csv('https://raw.githubusercontent.com/datasets/world-cities/refs/heads/main/data/world-cities.csv',delimiter=',')
# Crear una nueva columna con el nombre de la ciudad sin tildes y en minúscula
cities_df['name_adj'] = cities_df['name'].apply(lambda nombre: ''.join(c for c in unicodedata.normalize('NFD', nombre) if unicodedata.category(c) != 'Mn')).str.lower()

# Crear un diccionario para hacer búsquedas rápidas
ciudades_dict = dict(zip(cities_df['name_adj'], cities_df['name']))

# Definir función para estandarizar el nombres de las ciudades
def estandarizar_ciudad(nombre):

    # Quitar las tildes
    nombre = ''.join(c for c in unicodedata.normalize('NFD', nombre) if unicodedata.category(c) != 'Mn')

    if re.findall(r"barxelona",nombre):
        nombre = 'Barcelona'
    elif re.findall(r"nan|remote",nombre):
        nombre = 'otra'
        
    # Eliminar los espacios en blanco y colocar la cadena en minúscula
    nombre = nombre.strip().lower()

    return ciudades_dict.get(nombre, None)

In [280]:
# Limpiar los campos de Pais y Ciudad
data['pais'] = data['pais'].apply(estandarizar_pais)
data['ciudad'] = data['ciudad'].apply(estandarizar_ciudad)

In [ ]:
# Definir tasas de cambio a COP
tasa_cambio = {'USD':4153.28,
               'GBP': 5152.05,
               'CAD': 2885.65,
               'EUR': 4293.20,
               'AUD/NZD': 2604.10,
               'CHF':4565.48,
               'ZAR':224.47,
               'SEK':376.53,
               'HKD':533.11,
               'JPY':27.44}

In [ ]:
# Definir función que recibe como parámetros el monto y la moneda, y realiza el cambio a COP
def convertir_a_cop(monto, moneda):
    moneda = 'USD' if moneda == 'Other' else moneda 
    try:
        return monto * tasa_cambio[moneda]
    except KeyError:
        return None

In [ ]:
# Crear nuevos campos salario anual y compensaciones convirtiendo sueldos y compensaciones a Pesos Colombianos
data['salario_anual_cop'] = data.apply(lambda x: convertir_a_cop(x['salario_anual'],x['moneda']),axis=1).astype(float)
data['compesaciones_cop'] = data.apply(lambda x: convertir_a_cop(x['compensaciones'],x['moneda']),axis=1).astype(float)

In [287]:
# Crear nuevo campo salario anual mas compensaciones
data['salario_total_cop'] = data['salario_anual_cop'] + data['compesaciones_cop']

In [ ]:
data.to_csv('database_limpia.csv')